In [3]:
import json
from tqdm import tqdm, trange

In [ ]:
data_path_origin = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/gsm8k-train.json"
data_path_generate = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/disturbed_qa.jsonl"
output_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/gsm8k-train-disturbed.json"

prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request. Output each step in a separate line, and explicitly state the final answer after the final step following the format \"The answer is\".

### Instruction:
{}

### Response: 
Let’s think step by step."""

with open(data_path_origin, 'r') as f:
    origin_data = json.load(f)
    
with open(data_path_generate, 'r') as f:
    generate_data = f.readlines()
    generate_data = [json.loads(x) for x in generate_data]
    
result = []
for generate_data_item in generate_data:
    origin_query = origin_data[generate_data_item['id']]['prompt']
    if abs(len(generate_data_item['query']) - len(origin_query)) / len(origin_query) < 0.2:
        # generate_data_item['prompt'] = prompt.format(generate_data_item['query'])
        generate_data_item['prompt'] = generate_data_item['query']
        result.append(generate_data_item)
        
with open(output_path, 'w') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/home/nfs05/laizj/model/models--Qwen--Qwen2.5-Math-7B-Instruct/snapshots/ef9926d75ab1d54532f6a30dd5e760355eb9aa4d")
input_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/data/gsm8k-train.json"
output_path = "/home/nfs02/laizj/experiment/uncertainty_analysis/analysis_unknown/generated/generate_template_and_code_gsm8k-train.json"


prompt = """
Task Description:
Create:
A variable substitution template
A Python script generator that converts query responses to executable code
Input Format:
Query: [Original query with specific values]
COT Response: [Chain-of-thought response explaining the solution]
Output Requirements:
Template:
Replace concrete values in the query with <variable_name> placeholders
Maintain original structure while making it generic
Python Script:
Start with variable definitions matching the template placeholders
Convert the COT logic to executable Python code
Include a final output statement
Preserve variable naming consistency with the template

### Query:
A rectangle has a length of 3 inches and a width of 6 inches. A square has a width of 5 inches. What is the difference in area between the two shapes?

### Response:
The area of the rectangle is 3 x 6 = <<3*6=18>>18 sq in
The area of the square is 5 x 5 = <<5*5=25>>25 sq in
The difference is 25 - 18 = <<25-18=7>>7 sq in
#### 7

### Template:
A rectangle has a length of <length> inches and a width of <width> inches. A square has a width of <square_side> inches. What is the difference in area between the two shapes?

### Python Code:
```python
# Variable definitions
length = 3
width = 6
square_side = 5

# Calculations
rectangle_area = length * width
square_area = square_side * square_side
difference = square_area - rectangle_area

# Output
print(difference)
```

### Query:
{}

### Response:
{}
"""

with open(input_path, "r") as f:
    data = json.load(f)

results = []
for idx, item in enumerate(data):
    
    messages = [
    {"role": "system", "content": ""},
    {"role": "user", "content": prompt.format(item["prompt"], item["chosen"])}
    ]
    
    results.append({
        "id": idx,
        "query": item["prompt"],
        "prompt": tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True),
        "answer": item["answer"]
    })

with open(output_path, "w") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)